In [ ]:
from constants import set_cwd_to_project_root

set_cwd_to_project_root()

PAIR = "BTC/USDT:USDT"

In [100]:
# Load data using values set above
from freqtrade.data.history import get_datahandler

from freqtrade.enums import CandleType
from constants import DATA_DIR

data_handler = get_datahandler(
    datadir=DATA_DIR / "binance",
    data_format="feather",
)

TypeError: DataProvider.__init__() missing 1 required positional argument: 'exchange'

In [75]:
trades = data_handler.trades_load(PAIR, CandleType.FUTURES)

trades.head()

,open,high,low,close,volume,delta,amount,cost,price
2024-09-09 00:00:00,54839.6,54861.1,54728.6,54744.0,866.859,-233.681,866.825,4.750366e+07,54744.0
2024-09-09 00:05:00,54744.1,54938.3,54744.1,54922.6,836.499,-26.379,836.499,4.590194e+07,54922.6
2024-09-09 00:10:00,54922.6,55119.9,54850.4,54910.4,2419.798,-143.166,2419.798,1.330679e+08,54910.4
2024-09-09 00:15:00,54910.4,55149.6,54887.5,55079.8,893.877,-0.183,893.877,4.920216e+07,55079.8
2024-09-09 00:20:00,55079.8,55177.2,55040.5,55059.6,979.763,-72.007,979.763,5.399835e+07,55059.6


In [ ]:
candles = data_handler.ohlcv_load(PAIR, "5m", CandleType.FUTURES)

candles = candles[(candles["date"] <= trades["date"].max()) & (candles["date"] >= trades["date"].min())]

candles.head()

In [ ]:
from freqtrade.data.converter import populate_dataframe_with_trades
from collections import OrderedDict

# Initialize an empty OrderedDict for cached_grouped_trades
cached_grouped_trades = OrderedDict()

dataframe, cached_grouped_trades = populate_dataframe_with_trades(
    cached_grouped_trades=cached_grouped_trades,
    config={
        "timeframe": "5m", 
        "orderflow": {
            "cache_size": 1000,
            "max_candles": 1500,
            "scale": 0.5,
            "stacked_imbalance_range": 3,
            "imbalance_volume": 1,
            "imbalance_ratio": 3
        }
    },
    dataframe=candles,
    trades=trades
)

dataframe.set_index("date", inplace=True)

dataframe.head()

In [36]:
import plotly.graph_objects as go

# Create a candlestick chart
fig = go.Figure()

# Add candlestick trace
fig.add_trace(go.Candlestick(
    x=dataframe.index,
    open=dataframe['open'],
    high=dataframe['high'],
    low=dataframe['low'],
    close=dataframe['close'],
    name='Candlestick'
))

# Add bar chart trace to the right of the candles
fig.add_trace(go.Bar(
    x=dataframe["delta"],
    y=dataframe.index,
    orientation='h',
    name='Trade Volume',
    marker=dict(color=['rgba(255,0,0,0.5)' if d < 0 else 'rgba(0,255,0,0.5)' for d in dataframe["delta"]]),
    xaxis='x2',
    yaxis='y'
))

# Update layout to include the bar chart on the right
fig.update_layout(
    xaxis2=dict(
        title='Trade Volume',
        side='top',
        overlaying='x',
        range=[dataframe["delta"].min() * 1.1, dataframe["delta"].max() * 1.1],
        position=1
    ),
    yaxis=dict(
        domain=[0, 1]
    ),
    barmode='overlay'
)

# Update layout for better visibility
fig.update_layout(
    xaxis=dict(title='Time'),
    yaxis=dict(title='Price'),
    title='Order Heat Map'
)


# Update layout
fig.update_layout(
    title=f'{PAIR} OHLCV Data',
    xaxis_title='Date',
    yaxis_title='Price (USDT)',
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()

C:\Users\j10st\AppData\Local\Temp\ipykernel_64168\2727297936.py:17: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

